In [1]:
import requests
import pandas as pd
import re

In [2]:
dates = [['03-21', '03-22'], ['03-23', '03-24'],['03-28', '03-29'],['03-30','03-31'], ['04-06'], ['04-08']]
nums = [64,32,16,8,2,1]

all_rows = []
for i in range(len(dates)):
    for d in dates[i]:
        url = f'https://www.vegasinsider.com/college-basketball/odds/las-vegas/?date=2024-{d}'
        req = requests.get(url)
        games = req.text.split('\n')

        for j in range(len(games)):
            if 'data-moneyline' in games[j]: break
        for game in games[j:]:
            row = dict()
            teams = [m.group() for m in re.finditer(r'(?<=college-basketball/teams/)[a-z\-]+(?=/")', game)]
            row['team1'] = teams[0]
            row['team2'] = teams[1]

            odds = [m.group() for m in re.finditer(r'(?<="data-moneyline"\> )[\+\-0-9]+(?= <)', game)]
            num_odds = len(odds)
            row['team1odds'] = int(odds[num_odds//2-1])
            row['team2odds'] = int(odds[num_odds-1])
            all_rows.append(row)

names_df = pd.read_csv('data/MTeamSpellings.csv', encoding='cp1252')
odds_df = pd.DataFrame(all_rows)
odds_df = odds_df.merge(names_df, how = 'inner', left_on = 'team1', right_on=['TeamNameSpelling'])\
                 .drop(columns = ['team1', 'TeamNameSpelling'])\
                 .rename(columns = {'TeamID': 'team1'})\
                 .merge(names_df, how = 'inner', left_on = 'team2', right_on=['TeamNameSpelling'])\
                 .drop(columns = ['team2', 'TeamNameSpelling'])\
                 .rename(columns = {'TeamID': 'team2'})

def moneyline_to_return(moneyline):
    if moneyline > 0: return (moneyline - 100)/100
    return 100/-moneyline

odds_df['team1ret'] = odds_df['team1odds'].apply(moneyline_to_return)
odds_df['team2ret'] = odds_df['team2odds'].apply(moneyline_to_return)
odds_df[['team1', 'team2', 'team1ret', 'team2ret']].to_csv('odds.csv', index = False)